# Step 0. Parameter Setting

In [ ]:
import dtnmonitor_v2 as dtn
import os,sys


### P1. SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 
#dtn.interface = 'ens27f0.3061'
#dtn.interface = 'eth1'
dtn.interface = 'all'


### P3. Iperf3 SERVER IP 
## used in Step 2,3 : input iperf3 server's ip
iperf3_server="192.168.61.58"

### P4. Iperf3 THREADS
## used in Step 2,3 :input threads number
count = 8

test_time="20" # second

window_size="1M" # M,K

## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

### Show Disk and Interface
import netifaces
import os
import subprocess
import re

list=netifaces.interfaces()
print("==========================================================")
print('{:40s} {:20s} '.format("Network Interface","IP"))
print("==========================================================")
for inf in list:
    #print(inf)
    ff=netifaces.ifaddresses(inf)
    #print(ff)
    try:
        addr=ff[netifaces.AF_INET][0]['addr']
    except:
        continue
    print('{:40s} {:20s} '.format(inf,addr)) 

    
import getpass
user=getpass.getuser()

def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)

In [ ]:

print("==========================================================")
print('{:60s} '.format("Initial checking"))
print("==========================================================")

command="ps aux |grep 'bandw.py '|wc -l"
process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
ret=process.stdout.read().decode('utf8')

if int(ret) > 2: 
    print("System check ..... [Error]")
    print("Error!! Some process are running, please check first!!!")
    exec_print("ps -u "+ user +" ux |grep 'bandw.py '")
else : 
    print('Your Monitoring Network Interface ..... {:40s} '.format(dtn.interface))
    print("System check ..... [ok]")
    print("==========================================================")
    print('{:60s} '.format("Iperf3 Testing  ..."))
    print("==========================================================")
    test_cmd="iperf3 -w "+window_size+" -Z -t 5s -p 54000 -c "+ iperf3_server
    print(test_cmd)
    exec_print(test_cmd)

# Step 2. Test network transfer by memory

In [ ]:
command = ""
for i in range(count):
     command += "iperf3 -w "+window_size+" -Z -t "+test_time+"s -p 5400"+str(i)+ " -c "+ iperf3_server +" &"

os.system(command)

test_time=str(int(test_time)+20)

dtn.exec_command("sleep "+test_time,1)



## Step A . stop the Iperf3 server

In [ ]:
exec_print("killall -u "+user+" iperf3")

## Step B . check Iperf3 server status

In [ ]:
exec_print("ps -u "+ user +" ux |grep iperf3")